### Noise Gaussian

In [1]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   


root_dir_output_moderate = f'/workspace/data/output/moderate_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_moderate}/NLM'
dir_out_geonlm =  f'{root_dir_output_moderate}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_moderate}/BM3D'
dir_out_results =  f'{root_dir_output_moderate}/results'

In [ ]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   

import time


root_dir_output_moderate = f'/workspace/data/output/moderate_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_moderate}/NLM'
dir_out_geonlm =  f'{root_dir_output_moderate}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_moderate}/BM3D'
dir_out_results =  f'{root_dir_output_moderate}/results'


array_dir = read_directories(dir_images_general)

array_nln_moderate_filtereds = []

time_nlm = 0

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)

    m, n = img.shape

    # Ruído (σ em 0–255, sem alterar seus valores)
    ruidosa = add_moderate_noise_gaussian(img)

    # Clipa para [0,255]
    ruidosa = np.clip(ruidosa, 0, 255)

    # CPU/GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)  # data_range implícito 255
    img_gpu_noisse_gaussian = cp.array(img_noisse_gaussian_np)

    h_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)
    q_nlm_candidates = np.array([h_nlm + delta for delta in range(-50, 50, 1)])

    ini = time.time()
  
   
    # IMPORTANTE: passe a imagem de referência também em [0,255]
    img_filtered_nlm, nlm_h, psnr_nlm, ssim_nlm, score_nlm = select_best_h_using_adaptive_q(
        image=img,                          # agora está em [0,255] float32
        image_gpu=img_gpu_noisse_gaussian,  # idem
        q_nlm_candidates=q_nlm_candidates,
        f=4, t=7,
        alpha=0.5,
    )
    end = time.time()
   
    time_nlm += end - ini 
    dct = {
        'img_noisse_gaussian_np': img_noisse_gaussian_np,
        'img_filtered_nlm': img_filtered_nlm,
        'estimated_sigma_gaussian': estimated_sigma_gaussian_np,
        'nlm_h': nlm_h,
        'psnr_nlm': psnr_nlm,
        'ssim_nlm': ssim_nlm,
        'score_nlm': score_nlm,
        'file_name': file_name,
    }
    array_nln_moderate_filtereds.append(dct)

    #skimage.io.imsave(f'{dir_out_nlm}/{file_name}',np.clip(img_filtered_nlm, 0, 255).astype(np.uint8))
save_pickle(array_nln_moderate_filtereds, dir_out_results, "array_nln_moderate_filtereds.pkl")
print(f'time_nlm:{time_nlm}')

h = 129.00 | PSNR = 29.65 | SSIM = 0.8163 | Score = 55.64

[SELECTED] H = 129.00 | PSNR = 29.65 | SSIM = 0.8163 | SCORE = 55.64
h = 129.00 | PSNR = 32.46 | SSIM = 0.8558 | Score = 59.02

[SELECTED] H = 129.00 | PSNR = 32.46 | SSIM = 0.8558 | SCORE = 59.02
h = 129.00 | PSNR = 30.23 | SSIM = 0.7596 | Score = 53.10

[SELECTED] H = 129.00 | PSNR = 30.23 | SSIM = 0.7596 | SCORE = 53.10
h = 129.00 | PSNR = 31.33 | SSIM = 0.8118 | Score = 56.25

[SELECTED] H = 129.00 | PSNR = 31.33 | SSIM = 0.8118 | SCORE = 56.25
h = 129.00 | PSNR = 29.39 | SSIM = 0.7313 | Score = 51.26

[SELECTED] H = 129.00 | PSNR = 29.39 | SSIM = 0.7313 | SCORE = 51.26
h = 129.00 | PSNR = 30.10 | SSIM = 0.8459 | Score = 57.34

[SELECTED] H = 129.00 | PSNR = 30.10 | SSIM = 0.8459 | SCORE = 57.34
h = 129.00 | PSNR = 31.53 | SSIM = 0.7683 | Score = 54.18

[SELECTED] H = 129.00 | PSNR = 31.53 | SSIM = 0.7683 | SCORE = 54.18
h = 129.00 | PSNR = 33.62 | SSIM = 0.8722 | Score = 60.42

[SELECTED] H = 129.00 | PSNR = 33.62 | SSIM =

### GEONLM BM3D

In [ ]:
from geonlm_functions import run_geonlm_pipeline
import time
from Utils import (save_results_to_xlsx, load_pickle)

vetor = load_pickle(dir_out_results, "array_nln_moderate_filtereds.pkl")
from bm3d import bm3d, BM3DProfile
from skimage.restoration import estimate_sigma
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.morphology import disk

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

array_gnlm_bm3d_moderate_filtereds = []
for array in vetor:

    img_noisse_gaussian_np =array['img_noisse_gaussian_np']
    
    nlm_h =array['nlm_h']
    file_name =array['file_name']
    psnr_nlm =array['psnr_nlm']
    ssim_nlm =array['ssim_nlm']
    score_nlm =array['score_nlm']
    estimated_sigma_gaussian = array['estimated_sigma_gaussian']

    print(f'filename: {file_name}')
    print(f'psnr_nlm: {psnr_nlm}')
    print(f'ssim_nlm: {ssim_nlm}')
    print(f'score_nlm: {score_nlm}')

    
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)
    
    # Parâmetros
    f = 4
    t = 7
    nn = 10  

    ini = time.time()
    print("\n--- Executando Pipeline 512x512 com mult = 1.55 ---")
    mult = 1.55
    img_filtered_gnlm, h_gnlm, psnr_gnlm, ssim_gnlm, score_gnlm = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   
    skimage.io.imsave(f'{dir_out_geonlm}/{file_name}', img_filtered_gnlm.astype(np.uint8)) 
    end = time.time()
    time_geonlm = end - ini  

    img_bm3d = img.copy()

    if len(img_bm3d.shape) > 2:
        img_bm3d = skimage.color.rgb2gray(img_bm3d)
        img_bm3d = 255 * img_bm3d
    # Remove frames extras, se existirem (e.g. GIF com shape (1, H, W) ou (N, H, W, 3))
    if img_bm3d.ndim == 4:
        img_bm3d = img_bm3d[0]
    elif img_bm3d.ndim == 3 and img_bm3d.shape[2] != 3:
        img_bm3d = np.squeeze(img_bm3d)

    # Converte para escala de cinza, se necessário
    if img_bm3d.ndim == 3 and img_bm3d.shape[2] == 3:
        img_bm3d_gray = rgb2gray(img_bm3d)  # retorna float64 em [0, 1]
    else:
        img_bm3d_gray = img_bm3d.astype(np.float32) / 255.0  # já está em cinza
     # Agora está garantidamente em escala de cinza float [0,1]
    img_bm3d_gray = np.clip(img_bm3d_gray * 255, 0, 255).astype(np.uint8)
    
     # 1. Normaliza a imagem ruidosa para intervalo [0, 1]
    ruidosa_normalizada = img_noisse_gaussian_np.astype(np.float32) / 255.0

    sigma_est = estimate_sigma(ruidosa_normalizada, channel_axis=None)

    inicio = time.time()
    # 3. Define o perfil padrão do BM3D
    perfil_bm3d = BM3DProfile()
    fim = time.time() 
    time_bm3d = fim - inicio

    # 4. Aplica o BM3D com os argumentos definidos
    denoised = bm3d(
        ruidosa_normalizada,
        sigma_psd=sigma_est,
        profile=perfil_bm3d
    )
    denoised_sq = np.squeeze(denoised)   

    skimage.io.imsave(f'{dir_out_bm3d}/{file_name}',  np.clip(denoised_sq * 255, 0, 255).astype(np.uint8))

    
    psnr_bm3d = peak_signal_noise_ratio(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    ssim_bm3d = structural_similarity(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    score_bm3d = 0.5 * psnr_bm3d + 0.5 * (ssim_bm3d * 100)    
   

    dict = {     
        'nlm_h':nlm_h, 
        'h_gnlm':h_gnlm,       
        'estimated_sigma_gaussian':estimated_sigma_gaussian,      

        'ssim_nlm':ssim_nlm,
        'ssim_gnlm':ssim_gnlm,      
        'ssim_bm3d': ssim_bm3d,

        'psnr_nlm': psnr_nlm,
        'psnr_gnlm':psnr_gnlm,      
        'psnr_bm3d': psnr_bm3d,
        'time_geonlm':time_geonlm,
        'time_bm3d':time_bm3d,
        
        'score_nlm':score_nlm,   
        'score_gnlm':score_gnlm,      
        'score_bm3d': score_bm3d,   
       
        'file_name':file_name,

    }
    array_gnlm_bm3d_moderate_filtereds.append(dict)
save_pickle(array_gnlm_bm3d_moderate_filtereds, dir_out_results, "results_gaussian_gnlm_bm3d_moderate.pkl")
save_results_to_xlsx(array_gnlm_bm3d_moderate_filtereds, dir_out_results, 'gnlm_bm3d_moderate_filtereds.xlsx')